## Chapter 12: New Data Types in Julia

In this chapter we investigate how to create a new datatype, which is 1) quite easy to do in Julia and 2) can make life much easier if designed right.

### 12.1 Basics of a Composite type

The way to create a new type in julia is to create a `struct`.  The following makes a new type with two fields

In [ ]:
struct Mystruct 
  num::Integer 
  str::String
end

We can make an "object" of that type by calling the type like a function.  It must be called with the fields as arguments in the proper order:

In [ ]:
m=Mystruct(11,"hello")

The fields of a struct can be accessed via the . syntax:

In [ ]:
m.num

In [ ]:
m.str

In [ ]:
fieldnames(Mystruct)

In [ ]:
fieldnames(m)

Note that the function fieldnames is called with the type not an object the type. 

In [ ]:
m.num=15

structs are immutable. That is, after being created, they can't be altered. If you need one that is Mutable, add the keyword `mutable` to the front. 

In [ ]:
mutable struct MutableStruct
  a::Float64
  b::Integer
end

In [ ]:
s = MutableStruct(1,2)

In [ ]:
s.a=4.5

### 12.2: A Card datatype

We are going to do some simulations soon with playing cards.  We have the ranks and suits defined as follows.  The suits are the Unicode characters for the suits. 

In [ ]:
ranks = ['A','2','3','4','5','6','7','8','9','T','J','Q','K'];
suits = ['\u2660','\u2661','\u2662','\u2663']

And we will create a `Card` struct that contains two fields, a rank and a suit.  We can make both integers:

In [ ]:
struct Card
    rank::Integer
    suit::Integer
end

In [ ]:
c = Card(3,2)

This is a very boring looking 3 of hearts, so we use the following which will print out a Card much nicer:

In [ ]:
Base.show(io::IO, c::Card) = print(io, string(ranks[c.rank],suits[c.suit]))

We are creating a specific `show` function that is called whenever a `Card` object is printed or otherwise shown. 

In [ ]:
c = Card(3,2)

Let's expand the Card type.  It would be nice to also have a way of making a Card on a single integer.  (Later we will make an entire deck from the cards from 1 to 52).  The following does this:

In [ ]:
struct Card
  rank::Int
  suit::Int
  Card(r::Int,s::Int)=(1<=r<=13) ? ( (1<=s<=4) ? new(r,s) :
    throw(ArgumentError("The argument for suit must be between 1 and 4"))) :
    throw(ArgumentError("The argument for rank must be between 1 and 13")) 
  Card(i::Int) = !(1<=i<=52) ?
    throw(ArgumentError("The argument must be an integer between 1 and 􏰍52")) :
    i%13==0 ? new(13,div(i,13)) : new(i%13,div(i,13)+1) 
end

The error occurs because `Card` is immutable and we are trying to redefine it.  So restart the kernel and rerun this. Also, rerun the cell with the ranks an suits arrays declared as well as the `Base.show` function.

Also, we put in some tests on the type of inputs we can make.  First, the suits should satisfy $1\leq s\leq 4$ and the ranks should satisfy $1\leq s\leq13$.  We throw an error with a good error message if it doesn't work.

This shows that you can create two or more constructors for a type.  Some things with this:
* You must define a default Constructor that fills the fields in the order given. 
* The `new(r,s)` function is a way to call that default constructor.  

In [ ]:
Card(45)

In [ ]:
Card(73)

We also would like a set of cards and we're going to use an array to do this and call it a `Hand`. 

In [ ]:
struct Hand
    cards::Array{Card,1}
end

This creates a hand.

In [ ]:
h=Hand([Card(2,3),Card(12,1),Card(10,1),Card(10,4),Card(5,2)])

And it's a good idea to define a `Base.show` for a `Hand`.  This just makes a string that is calling the `Card` version of `Base.show` and then joining the cards. 

In [ ]:
Base.show(io::IO,h::Hand) = print(io, string("[",join(h.cards,", "),"]"))

In [ ]:
h

#### 12.3: Polynomial Datatype

In this section, we look at a polynomial datatype.  We will look at creating polynomials, adding them, evaluating them and plotting this.  First, we define a polynomial with integer coefficients.  Don't run this though:

In [ ]:
struct Polynomial
  coeffs::Vector{Int64}
end

This will define a polynomial with integer coefficients, but it will be a pain, if we need to define different polynomials, with different types of coefficients.  We can create a Polynomail with any number type using:

In [ ]:
struct Polynomial{T <: Number}
  coeffs::Vector{T}
end

In [ ]:
poly1 = Polynomial([1,2,3])

which again is a terrible way to represent a polynomial, so let's create another `Base.show` for a polynomial

In [ ]:
function Base.show(io::IO, p::Polynomial)
  str = ""
  for i = 1:length(p.coeffs)
      str = string(str,p.coeffs[i],"x^",i-1,i<length(p.coeffs) ? "+" : "")
  end
  print(io, str)
end

In [ ]:
p = Polynomial([1,2,3])

#### Create different polynomials:

In [ ]:
poly1=Polynomial([1,2,3])
poly2=Polynomial([1.0,2.0,3.0])
poly3=Polynomial([2//3,3//4,5//8])
poly4=Polynomial([im,2+0im,3-2im,-im])
poly5=Polynomial([n for n=1:6])

In [ ]:
poly4

Evaluate a polynomial at a number using Horner's method:

In [ ]:
function eval(p::Polynomial,x::Number)
  result = p.coeffs[end] 
  for i=length(p.coeffs)-1:-1:1
    result = x*result+p.coeffs[i]
  end
  result
end

In [ ]:
eval(poly5,3)

In [ ]:
eval(poly1,0)

#### Adding two polynomials:

To add two polynomails, we first need to import `Base.+`:

In [ ]:
import Base.+

In [ ]:
function +(p1::Polynomial{T},p2::Polynomial{S}) where {T <: Number, S <: Number}
  Polynomial(p1.coeffs+p2.coeffs) 
end

In [ ]:
poly1+poly2

#### Plotting a polynomial

In [ ]:
using Plots,RecipesBase

In [ ]:
@recipe function f(poly::Polynomial,xmin::Number=-2,xmax::Number=2) 
  xpts = LinRange(xmin,xmax,200)
  ypts = map(x->eval(poly,x),xpts)
  xpts,ypts
end

In [ ]:
plot(poly1)

In [ ]:
@recipe function f(poly::Polynomial,xmin::Number=-2,xmax::Number=2) 
  legend -->  false
  linecolor -->  :green
  xpts = LinRange(xmin,xmax,200)
  ypts = map(x->eval(poly,x),xpts)
  xpts,ypts
end

In [ ]:
plot(poly1)

In [ ]:
plot(poly1,-5,5)

#### 12.4: Develop a Root datatype

Recall that when we developed Newton's method in Chapter 10, we weren't sure if a function didn't have a root (or Newton's method didn't find it.)  Here we will create a type called `Root` that will store all of the important information about the results of Newton's method and return a `Root` object, instead of just a number.

We're going to create a `Root` struct that stores a lot of information about the root. The fields are:

* `root`, the approximation to the root
* `x_eps`, an approximate error to the root
* `f_eps`, an estimate of the function value at the approximate root
* `num_steps`, the number of steps of Newton's method
* `converged`, a boolean on whether or not Newton's method converged. 
* `max_steps`, the maximum number of steps specified. 

In [ ]:
struct Root
  root::Float64
  x_eps::Float64
  f_eps::Float64
  num_steps::Int
  converged::Bool
  max_steps::Int
end

Here's a version of Newton's method that returns a `Root` object:

In [ ]:
using ForwardDiff

function newton(f::Function,  x0::Real)
  local n=0
  local dx=f(x0)/ForwardDiff.derivative(f,x0)
  while abs(dx)>1e-6
    x0 = x0-dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    n += 1 
    if n==10  # if too many steps are taken, break out of the while loop
      return Root(x0,dx,f(x0),n,false,10)
    end
  end
  Root(x0,dx,f(x0),n,true,10)
end

In [ ]:
newton(x->x^2-2,1)

This isn't much of a helpful struct, because you might not remember what each parameter is. 

In [95]:
function Base.show(io::IO,r::Root)
  if(r.converged)
    str = string("The root is approximately x̂ = $(r.root)\n")
    str = string(str,"An estimate for the error is ",r.x_eps,"\n")
    str = string(str,"with f(x̂) = ",r.f_eps,"\n")
    str = string(str,"which took ",r.num_steps," steps")
  else
    str = string("The root was not found within ",r.max_steps," steps.\n");
    str = string(str,"Currently, the root is approximately x̂ = ",r.root," \n")
    str = string(str,"An estimate for the error is ",r.x_eps,"\n")
    str = string(str,"with f(x̂) = ",r.f_eps,"\n")
  end
  print(io,str)
end

Let's rerun this now and result is much clearer.

In [93]:
r = newton(x->x^2-2,1)

StackOverflowError: StackOverflowError:

And if we put in a function that doesn't converge, will tell us as well.

In [ ]:
r = newton(x->x^2+1,2)